### In this notebook the differnce between available and annotated data is taken



In [2]:
import pydicom
import glob
import pandas as pd
from datetime import datetime

In [3]:
lesions = glob.glob("/data/oleksii/alta-ai-orthanc-backup/2023_07_26_full/prostate_lesion/*/*/seg.dcm")
len(lesions)

1444

In [4]:
dataset = pydicom.dcmread(lesions[0])
# dataset

In [6]:
meta_info = []
for seg in lesions:
    orthancID = seg.split("/")[-2]
    dataset = pydicom.dcmread(seg)
    StudyInstanceUID = dataset[(0x0020, 0x000d)].value
    meta_info.append({"orthancID": orthancID, "StudyInstanceUID": StudyInstanceUID})
avaliable_lesions = pd.DataFrame(meta_info)

In [8]:
avaliable_lesions

,orthancID,StudyInstanceUID
0,53be6c8b-5dfd882b-a4b4ff48-157915bd-4f068ee4,1.2.276.0.38.1.1.1.10507.20211221073720.50687679
1,bfb3bed5-3dba8e74-98ead390-924b83a3-148809da,1.2.276.0.115.1.110.780.1671717264.1671717264533
2,7ca8800e-cf9ac339-d41d2a2f-184f181a-99c2ce74,1.3.12.2.1107.5.2.30.59050.3000002301190627332...
3,44bce4a0-a4aaed96-d37ea0b0-a6a9d160-81a2a7b8,1.2.276.0.115.1.110.1558.1662543211.1662543211020
4,714e2721-09d88d59-abcc34ab-43bdcae1-486acbbe,1.3.12.2.1107.5.2.19.145132.300000230223063417...
...,...,...
1439,153a0a94-42c809c3-9b4997bc-d03635a1-8cfddc13,1.2.276.0.115.1.110.1556.1645783272.1645783272321
1440,ef289f2f-e37ed5ed-e458e281-e1319cd3-532036ab,1.3.12.2.1107.5.2.19.145132.300000190321064832...
1441,ed6c2a5b-bc4f9229-9cec0a5f-897baaed-5ebde8ef,1.3.12.2.1107.5.2.19.145132.300000210514065023...
1442,8d2d8a0b-92279ecb-a1863014-f329051c-b6047c8b,1.3.12.2.1107.5.2.19.145132.300000190328071300...


In [9]:
df_befund = pd.read_csv("meta_pacs_20230719_befund.csv", sep=";")

In [60]:
filtered = df_befund[~df_befund["StudyInstanceUID"].isin(avaliable_lesions["StudyInstanceUID"])]
upload_batch = filtered[~filtered['Befundtext'].isna()]
upload_batch = upload_batch.drop(columns="PatientName")

In [61]:
now = datetime.now().strftime("%Y%m%d")
upload_batch.to_csv(f'/home/oleksii/projects/ohif-orthanc-postgres-docker/upload_batch_{now}.csv', index=False, sep=";")

,InstitutionName,PatientAge,PatientBirthDate,PatientID,PatientName,PatientSize,PatientWeight,StudyDate,StudyInstanceUID,Info,Befundtext,Diagnose
0,Gemeinschaftspraxis NURAMED GbR,057Y,19651014,C178423,Schmitz^Wolfgang,"1,88",80,20230329.0,1.2.276.0.33.1.0.4.192.100.90.184.20230329.112...,mit Biopsie,\ndie histologische Untersuchung der MRT-geste...,Prostatakrebs
1,ALTA Klinik,064Y,19580818,41372,Karimyan^Artyom,"1,8",100,20230322.0,1.3.12.2.1107.5.2.19.145132.300000230322072211...,mit Biopsie,\ndie histologische Untersuchung der MRT-geste...,Prostatakrebs
2,ALTA Klinik,069Y,19530610,2240,Wöstmann^Norbert,"1,9",97,20230210.0,1.3.12.2.1107.5.2.19.145132.300000230210071018...,mit Biopsie,\ndie histologische Untersuchung der MRT-geste...,Prostatitis
3,Praxis Am Berg 51465 Berg. Gladbach,058Y,19640919,NX_250883,MAJONICA^INGO,"1,84",82,20230113.0,1.2.276.0.18.14.200.2.0.0.2.20230103.103616.57.11,mit Biopsie,\ndie MRT-gesteuerte Biopsie des rechten Prost...,"Prostatakrebs, Prostatitis"
4,ALTA Klinik,048Y,19740708,3261,Baftiri^Ariton,"1,89",90,20230612.0,1.3.12.2.1107.5.2.19.145132.300000230612072531...,mit Biopsie,\ndie MRT-gesteuerte und gezielte Biopsie des ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13009,NaN,057Y,19650219,4432831,Gräwe^Gerd,NaN,68,20221222.0,1.2.276.0.24.319.42991976.4.0.1,mit Biopsie,Im rechten Prostatalappen: Gezielt Transiti...,Prostatakrebs
13013,ALTA Klinik,058Y,19640214,29982,Ten Eicken^Thomas,"1,85",91,20230202.0,1.3.12.2.1107.5.2.19.145132.300000230202072025...,mit Biopsie,wir übersenden Ihnen in der Anlage den histopa...,NaN
13014,DieRadiologen DA-Weiterstadt,NaN,19590819,211975,Beck^Harald,NaN,NaN,20230127.0,1.2.276.0.50.6560656.4226990568504989288615721...,mit Biopsie,die histologische Untersuchung der MRT-gesteue...,Prostatakrebs
13015,ALTA Klinik,065Y,19570515,13375,Schulze^Manfred,"1,8",108,20230202.0,1.3.12.2.1107.5.2.19.145132.300000230202072025...,mit Biopsie,ergab die histologische Untersuchung der MRT-g...,NaN
